# Incompressible Model

This script demonstrates how to compute the effective Hamiltonian of AB-type copolymers for the incompressible model

### 1. Formulation
* The functional represention of the canonical partition function is:
\begin{align}
\mathcal{Z} \propto \int \{\mathcal{D}\Omega_i\} \exp(-\beta H[\{\Omega_i\}]),
\end{align}
where $\{\Omega_i\}$ is a set of all auxiliary fields, and $\{\mathcal{D}\Omega_i\}$ are functional integrals over all auxiliary fields.
* There are $M$ distinct monomer types.
* ${\alpha_p}=\frac{N_p}{N}$ is the total chain length of $p$-chain.
* Total volume is $V=\frac{nN}{\rho_0}$, and total volume fraction occupied by type $p$ is $V_p=\frac{n_pN_p}{\rho_0}$.
* The volume fraction occupied by type $p$ is $\bar{\phi}_p=\frac{V_p}{V}$. 
* $C=\frac{\rho_0 R_0^3}{N}$ is dimensionless polymer chain number density parameter, and the invariant polymerization index $\bar{N}=C^2$.
* $O$ is the orthogonal matrix of $PXP$ with $OO^T=M$, where $P$ is a projection matrix and $X$ is $\chi$ matrix.
* ${\bf s}^T = \frac{{\bf e}^T X}{M}$ and ${\bf S} = \frac{O^T{\bf s}}{M}$.
* The effective Hamiltonian $H$ is written as in 'per monomer' unit:
\begin{align}
\beta H &= -\sum_p n_p\log{Q_p} +
\rho_0\int d{\bf r}\left[
\sum_{i=1}^{M-1} \frac{M\Omega_i^2}{2\lambda_i} +
\sum_{i=1}^{M-1} \frac{\Omega_i S_i}{\lambda_i}
-\Omega_M + \frac{1}{2M}\left({\bf s}^T{\bf e}-\sum_{i=1}^{M-1} \frac{S_i^2}{\lambda_i}\right) \right],  \ \ \ \ (\textrm{per monomer unit})
\end{align}

* The effective Hamiltonian $H$ can be rewritten in 'per chain' unit:
\begin{align}
N\Omega_i &\rightarrow \Omega_i, \ \ N\lambda_i \rightarrow \lambda_i, \ \ N{\bf s} \rightarrow {\bf s}, \ \ N S_i \rightarrow S_i, \\
\frac{\beta H}{CV/R_0^3} &= -\sum_p \frac{\bar{\phi}_p}{\alpha_p}\log{Q_p} +
\frac{1}{V}\int d{\bf r}\left[
\sum_{i=1}^{M-1} \frac{M\Omega_i^2}{2\lambda_i} +
\sum_{i=1}^{M-1} \frac{\Omega_i S_i}{\lambda_i} 
-\Omega_M + \frac{1}{2M}\left({\bf s}^T{\bf e}-\sum_{i=1}^{M-1} \frac{S_i^2}{\lambda_i}\right) \right].  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The relations between the auxiliary potential fields $\{\Omega_i\}$ and monomer potential fields $\{W_i\}$ are
\begin{align}
\left[\begin{array}{cc} 
W_1({\bf r})\\
W_2({\bf r})\\
\cdots      \\
W_M({\bf r})\\
\end{array}\right]&=
O
\left[\begin{array}{cc} 
\Omega_1({\bf r}) \\ 
\Omega_2({\bf r}) \\ 
\cdots            \\
\Omega_M({\bf r}) \\ 
\end{array}\right],
\end{align}

#### Example 1) AB-type polymeric systems

* The effective Hamiltonian $H$ is written as:
\begin{align}
\beta H &= -\sum_p n_p\log{Q_p} +
\rho_0\int d{\bf r}\left[\frac{\Omega_-^2({\bf r})}{\chi} -\Omega_+({\bf r}) + \frac{\chi}{4}\right],  \ \ \ \ (\textrm{per monomer unit}) \\
\frac{\beta H}{CV/R_0^3} &= -\sum_p \frac{\bar{\phi}_p}{\alpha_p}\log{Q_p} +
\frac{1}{V}\int d{\bf r}\left[\frac{\Omega_-^2({\bf r})}{\chi N} -\Omega_+({\bf r}) + \frac{\chi N}{4}\right].  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The functional derivative $\frac{\delta H}{\delta \Omega_-({\bf r})}$ is 
\begin{align}
\frac{\beta}{\rho_0}\frac{\delta H}{\delta \Omega_-({\bf r})} &= \Phi_-({\bf r}) + \frac{2}{\chi}\Omega_-({\bf r}),  \ \ \ \ (\textrm{per monomer unit}) \\
\frac{\beta}{C/R_0^3}\frac{\delta H}{\delta \Omega_-({\bf r})} &= \Phi_-({\bf r}) + \frac{2}{\chi N}\Omega_-({\bf r}).  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The functional derivative $\frac{\delta H}{\delta \Omega_+({\bf r})}$ is 
\begin{align}
\frac{\beta}{\rho_0}\frac{\delta H}{\delta \Omega_+({\bf r})} &= \Phi_+({\bf r}) - 1,  \ \ \ \ (\textrm{per monomer unit}) \\
\frac{\beta}{C/R_0^3}\frac{\delta H}{\delta \Omega_+({\bf r})} &= \Phi_+({\bf r}) - 1.  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The relations between the auxiliary fields and monomer potential fields are
\begin{align}
\left[\begin{array}{cc} 
W_A({\bf r})\\
W_B({\bf r})
\end{array}\right]&=
O
\left[\begin{array}{cc} 
\Omega_-({\bf r}) \\ 
\Omega_+({\bf r}) 
\end{array}\right], \\
\left[\begin{array}{cc} 
\phi_A({\bf r})\\
\phi_B({\bf r})
\end{array}\right]&=
O
\left[\begin{array}{cc} 
\Phi_-({\bf r}) \\ 
\Phi_+({\bf r}) 
\end{array}\right],
\end{align}
where $O=\left[\begin{array}{cc} 
1 & 1 \\ 
-1 & 1
\end{array}\right]$.

References:
* [(2014) A multi-species exchange model for fully fluctuating polymer field theory simulations](https://aip.scitation.org/doi/pdf/10.1063/1.4900574)
* [(2025) Polymer Field Theory for Multimonomer Incompressible Models: Symmetric Formulation and ABC Systems](https://doi.org/10.1021/acs.macromol.4c02636)

### 2. Compute coefficients for computing Hamiltonian and functional derivatives

* This example uses pre-written `polymer_field_theory.py` located in \${PYTHON\_SITE\_PACKAGES}/polymerfts

* ```class polymerfts.SymmetricPolymerTheory``` computes the coefficents $a_i$, $b_i$, $U_{ref}$, $\frac{\partial a_i}{\partial \chi_{ab} N}$, $\frac{\partial b_i}{\partial \chi_{ab} N}$, and $\frac{\partial U_{ref}}{\partial \chi_{ab} N}$ for given parameters in 'per chain' unit.
\begin{align}
\frac{\beta H}{CV/R_0^3} &= -\sum_p \frac{\bar{\phi}_p}{\alpha_p}\log{Q_p} +
\frac{1}{V}\int d{\bf r}\left[
\sum_{i=1}^{M} a_i \Omega_i^2({\bf r}) + \sum_{i=1}^{M} b_i \Omega_i({\bf r}) + U_{ref}\right].  \ \ \ \ (\textrm{per chain unit})
\end{align}

* The functional derivative w.r.t $\Omega_i({\bf r})$ is
\begin{align}
\frac{\beta}{C/R_0^3}\frac{\delta H}{\delta \Omega_i({\bf r})} &= \Phi_i({\bf r}) +
2 a_i \Omega_i({\bf r}) + b_i.  \ \ \ \ (\textrm{per chain unit})
\end{align}
where $\Phi_i({\bf r})= \sum_j O_{ji}\phi_j({\bf r})$

* The functional derivative w.r.t $\chi_{ab} N$ is
\begin{align}
\frac{\beta}{CV/R_0^3}\frac{\partial \beta H}{\partial \chi_{ab} N} &= 
\frac{1}{V}\int d{\bf r}\left[
\sum_{i=1}^{M} \frac{\partial a_i}{\partial \chi_{ab} N} \Omega_i^2({\bf r}) + \sum_{i=1}^{M} \frac{\partial b_i}{\partial \chi_{ab} N} \Omega_i({\bf r}) + \frac{\partial U_{ref}}{\partial \chi_{ab} N}\right].  \ \ \ \ (\textrm{per chain unit})
\end{align}

* In polymerfts.SymmetricPolymerTheory, 'matrix_a' $=O$, and 'matrix_a_inv' $=\frac{1}{M}O^T$.

##### 1) Import modules and set environment variables

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import polymerfts

# OpenMP environment variables
os.environ["MKL_NUM_THREADS"] = "1"  # always 1
os.environ["OMP_STACKSIZE"] = "1G"
os.environ["OMP_MAX_ACTIVE_LEVELS"] = "1"  # 0, 1
os.environ["OMP_NUM_THREADS"] = "2"  # 1 ~ 4

# GPU environment variables
os.environ["LFTS_GPU_NUM_BLOCKS"]  = "256"
os.environ["LFTS_GPU_NUM_THREADS"] = "256"

monomer_types = ["A","B"]
chi_n = {"A,B":20}

# Polymer field theory for Multimonomer system
mpt = polymerfts.SymmetricPolymerTheory(monomer_types, chi_n, zeta_n=None)

# # Print Eigenvalues and Eigenvectors of PXP matrix
# print("------------")
# print("Eigenvalues: ", mpt.eigenvalues)
# print("Eigenvectors [v1, v2, ...] :\n\t", str(mpt.matrix_o).replace("\n", "\n\t"))
# print("Mapping matrix A:\n\t", str(mpt.matrix_a).replace("\n", "\n\t"))
# print("Inverse of A:\n\t", str(mpt.matrix_a_inv).replace("\n", "\n\t"))

# # # The numbers of real and imaginary fields, respectively
# M = len(monomer_types)
# R = len(mpt.aux_fields_real_idx)
# I = len(mpt.aux_fields_imag_idx)
# print("Number of real fields: ", R)
# print("Number of imaginary fields: ", I)

One of eigenvalues is zero for given chiN values.
------------ Polymer Field Theory for Multimonomer System------------
Eigenvalues:
	 [-20.   0.]
Eigenvectors [v1, v2, ...] :
	 [[ 1.  1.]
	 [-1.  1.]]
Mapping matrix A:
	 [[ 1.  1.]
	 [-1.  1.]]
Real Fields:  [0]
Imaginary Fields:  [1]
In Hamiltonian:
	reference energy:  5.0
	coefficients of int of mu(r)/V:  [-0. -1.]
	coefficients of int of mu(r)^2/V:  [0.05 0.  ]
	dH_ref/dχN:  {'A,B': 0.24999999999053554}
	d(coef of mu(r))/dχN:  {'A,B': array([0., 0.])}
	d(coef of mu(r)^2)/dχN:  {'A,B': array([-0.0025,  0.    ])}


##### 2) Test conversion between the auxiliary potential fields and monomer potential fields

\begin{align}
\left[\begin{array}{cc} 
W_A({\bf r})\\
W_B({\bf r})
\end{array}\right]&=
O
\left[\begin{array}{cc} 
\Omega_-({\bf r}) \\ 
\Omega_+({\bf r}) 
\end{array}\right], \\
\end{align}

In [2]:
nx = [2,2]                         # number of grid points in each direction
lx = [2.0, 2.0]                    # length of the box in each direction
stat_seg_lengths = {"A":1.0,       # statistical segment lengths
                    "B":1.0}        
ds = 0.01                          # contour step interval

np.random.seed(0)  # For reproducibility

omega = np.random.rand(len(monomer_types), np.prod(nx))
print("Initial auxiliary potential fields (omega):\n", omega)

# Convert auxiliary potential fields to monomer potential fields
w = mpt.to_monomer_fields(omega)
print("Converted monomer potential fields (w):\n", w)

# Convert monomer potential fields into auxiliary potential fields
omega = mpt.to_aux_fields(w)
print("Converted back to auxiliary potential fields (omega):\n", omega)

Initial auxiliary potential fields (omega):
 [[0.5488135  0.71518937 0.60276338 0.54488318]
 [0.4236548  0.64589411 0.43758721 0.891773  ]]
Converted monomer potential fields (w):
 [[ 0.9724683   1.36108348  1.04035059  1.43665618]
 [-0.1251587  -0.06929525 -0.16517616  0.34688982]]
Converted back to auxiliary potential fields (omega):
 [[0.5488135  0.71518937 0.60276338 0.54488318]
 [0.4236548  0.64589411 0.43758721 0.891773  ]]


##### 3) Prepare computation

In [3]:
# Select platform ("cuda" or "cpu-mkl") for real-valued simulations
reduce_gpu_memory_usage = False
factory = polymerfts.PlatformSelector.create_factory("cpu-mkl", reduce_gpu_memory_usage, "real")
factory.display_info()

# Create an instance for computation box
cb = factory.create_computation_box(nx, lx) 
# Create an instance for molecule information with block segment information and chain model ("continuous" or "discrete")
molecules = factory.create_molecules_information("continuous", ds, stat_seg_lengths)

# Add AB diblock copolymers
volume_fraction = 1.0
blocks = [["A", 0.7, 0, 1],   # monomer type, statistical segment length, start index, end index
          ["B", 0.3, 1, 2]]
molecules.add_polymer(volume_fraction, blocks)

# Optimizer to avoid redundant computations
aggregate_propagator_computation = True
propagator_computation_optimizer = factory.create_propagator_computation_optimizer(molecules, aggregate_propagator_computation)
propagator_computation_optimizer.display_blocks()
propagator_computation_optimizer.display_propagators()

# Create a solver
solver = factory.create_pseudospectral_solver(cb, molecules, propagator_computation_optimizer)

==================== MKL Version ====================
Major version:           2021
Minor version:           0
Update version:          4
Product status:          Product
Build:                   20210904
Platform:                Intel(R) 64 architecture
Processor optimization:  Intel(R) Architecture processors
--------------- Blocks ---------------
Polymer id, left key:
	aggregated, (left, right) is_junction, (left, right) n_segment, right key, n_repeat, {v, u} list

0, A:
	 X, (X, O), (70, 70), (B30)A, 1, {0,1}

0, B:
	 X, (X, O), (30, 30), (A70)B, 1, {2,1}
--------------- Propagators ---------------
Key:
	height, aggregated, max_n_segment, # dependencies, junction_ends
B:
	 0, X, 30, 0, {30}, 
A:
	 0, X, 70, 0, {70}, 
(B30)A:
	 1, X, 70, 1, {}, 
(A70)B:
	 1, X, 30, 1, {}, 
Total number of modified diffusion equation steps (time complexity) to compute propagators: 200
Total number of steps after optimizing computation : 200
Computational cost reduction (higher is better) : 0 %


##### 4) Compute the Hamiltonian and functional derivatives

In our implementation, there is a negative sign in front of the functional derivative w.r.t. an imaginary field. This is because we find the minimum and the maximum values for the real and imaginary fields, respectively.

Example)

```python
# Compute δH/δΩ
h_deriv = mpt.compute_func_deriv(omega, phi, [0 ,1])
```

\begin{align}
h\_deriv[0] &=\frac{\beta}{C/R_0^3}\frac{\delta H}{\delta \Omega_-({\bf r})} \\
h\_deriv[1] &=-\frac{\beta}{C/R_0^3}\frac{\delta H}{\delta \Omega_+({\bf r})}
\end{align}


In [4]:
# Compute Propagators (q) and single partition function (Q)
solver.compute_propagators({"A":w[0], "B":w[1]})

# Compute ensemble average concentration (phi)
solver.compute_concentrations()

# Get the ensemble average concentration for each monomer type
phi = {}
phi["A"] = solver.get_total_concentration("A")
phi["B"] = solver.get_total_concentration("B")

# Calculate Hamiltonian
total_partitions = [solver.get_total_partition(p) for p in range(molecules.get_n_polymer_types())]
hamiltonian_with_const = mpt.compute_hamiltonian(molecules, omega, total_partitions, include_const_term=True)
hamiltonian_without_const = mpt.compute_hamiltonian(molecules, omega, total_partitions, include_const_term=False)

# Compute δH/δΩ
h_deriv = mpt.compute_func_deriv(omega, phi, [0 ,1])

# Compute dH/dχN
dH = mpt.compute_h_deriv_chin(chi_n, omega)

print("\nHamiltonian with const term:\n", hamiltonian_with_const)
print("Hamiltonian without const term:\n", hamiltonian_without_const)
print("\nh_deriv[0] = dH/dΩ-:\n", h_deriv[0])
print("phi_A - phi_B + 2Ω-/chiN:\n", phi["A"] - phi["B"] + 2/chi_n["A,B"] * omega[0])

print("\nh_deriv[1] = -dH/dΩ+:\n", h_deriv[1])
print("phi_A + phi_B - 1:\n", phi["A"] + phi["B"] - 1)
print("\ndH/dχN:\n", dH)


Hamiltonian with const term:
 5.248221717906545
Hamiltonian without const term:
 0.2482217179065448

h_deriv[0] = dH/dΩ-:
 [0.51928897 0.42826267 0.49822666 0.39538665]
phi_A - phi_B + 2Ω-/chiN:
 [0.51928897 0.42826267 0.49822666 0.39538665]

h_deriv[1] = -dH/dΩ+:
 [-0.13043127  0.06986617 -0.09096665  0.15153175]
phi_A + phi_B - 1:
 [ 0.13043127 -0.06986617  0.09096665 -0.15153175]

dH/dχN:
 {'A,B': 0.2490794290764395}
